In [2]:
!pip install gym

/Users/sangilmo/.zshenv:source:1: no such file or directory: /Users/sangilmo/.zsh/BlaCk-Void.zshenv
     |████████████████████████████████| 1.6 MB 851 kB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 1.5 MB/s eta 0:00:01
Failed to build gym
ERROR: spyder 4.1.4 requires pyqt5<5.13; python_version >= "3", which is not installed.
ERROR: spyder 4.1.4 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 1.5.0
    Uninstalling cloudpickle-1.5.0:
      Successfully uninstalled cloudpickle-1.5.0
    Running setup.py install for gym ... done


## Q-Network

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf

import random
import gym

In [9]:
env = gym.make('CartPole-v1')

In [10]:
n_input = env.observation_space.shape[0] #4
n_actions = env.action_space.n #2
n_output = n_actions

n_hidden1 = 100
n_hidden2 = 50

In [12]:
tf.disable_eager_execution()
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])

In [13]:
weights = {
    'hidden1' : tf.Variable(tf.random_normal([n_input, n_hidden1], stddev = 0.01)),
    'hidden2' : tf.Variable(tf.random_normal([n_hidden1, n_hidden2], stddev = 0.01)),
    'output' : tf.Variable(tf.random_normal([n_hidden2, n_output], stddev = 0.01))
}

biases = {
    'hidden1' : tf.Variable(tf.random_normal([n_hidden1], stddev = 0.01)),
    'hidden2' : tf.Variable(tf.random_normal([n_hidden2], stddev = 0.01)),
    'output' : tf.Variable(tf.random_normal([n_output], stddev = 0.01))
}

In [14]:
def Qnet(x, weights, biases):
    hidden1 = tf.add(tf.matmul(x, weights['hidden1']), biases['hidden1'])
    hidden1 = tf.nn.relu(hidden1)
    
    hidden2 = tf.add(tf.matmul(hidden1, weights['hidden2']), biases['hidden2'])
    hidden2 = tf.nn.relu(hidden2)
    
    output = tf.add(tf.matmul(hidden2, weights['output']), biases['output'])
    return output

In [15]:
Qpred = Qnet(x, weights, biases)
loss = tf.reduce_mean(tf.square(y - Qpred))

LR = 0.001
optm = tf.train.AdamOptimizer(LR).minimize(loss)

In [17]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [23]:
state = env.reset()
print(state)

state = np.reshape(state, [1, 4])
print(state)

Q = sess.run(Qpred, feed_dict={x:state})
print(Q)

[ 0.02054273 -0.04727981 -0.04601002 -0.00252249]
[[ 0.02054273 -0.04727981 -0.04601002 -0.00252249]]
[[-0.0014436   0.01629434]]


In [24]:
gamma = 0.9

for episode in range(501):
    
    done = False
    state = env.reset()
    
    count = 0
    
    while not done:
        
        count += 1
        state = np.reshape(state, [1, 4])
        Q = sess.run(Qpred, feed_dict = {x:state})
        
        epsilon = 0.1
        if np.random.uniform() < epsilon:
            a = env.action_space.sample()
        else:
            a = np.argmax(Q)
            
        next_state, reward, done, _ = env.step(a)
        
        if done:
            reward = -200
            Q[0][a] = reward
        else:
            next_state = np.reshape(next_state, [1, n_input])
            next_Q = sess.run(Qpred, feed_dict = {x:next_state})
            Q[0][a] = reward + gamma*np.max(next_Q)
        
        sess.run(optm, feed_dict = {x: state, y: Q})
        state = next_state
    
    if episode % 100 == 0:
        print("Episode: {} steps: {}".format(episode, count))

env.close()

Episode: 0 steps: 9
Episode: 100 steps: 20
Episode: 200 steps: 49
Episode: 300 steps: 29
Episode: 400 steps: 63
Episode: 500 steps: 110


In [32]:
state = env.reset()

done = False

while not done:
    env.render()
    
    state = np.reshape(state, [1, n_input])
    Q = sess.run(Qpred, feed_dict={x:state})
    action = np.argmax(Q)
    
    next_state, reward, done, _ = env.step(action)
    state = next_state
    
    
env.close()

## DQN

Buffer (Experience Replay)를 사용하여 업데이트 한다. -> 더 효율적. 

In [55]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf

import random
from collections import deque
import gym
%matplotlib inline

In [56]:
env = gym.make('CartPole-v1')

In [57]:
n_input = env.observation_space.shape[0] #4
n_actions = env.action_space.n #2
n_output = n_actions

n_hidden1 = 100
n_hidden2 = 50

In [58]:
tf.disable_eager_execution()
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])

In [59]:
weights = {
    'hidden1' : tf.Variable(tf.random_normal([n_input, n_hidden1], stddev = 0.01)),
    'hidden2' : tf.Variable(tf.random_normal([n_hidden1, n_hidden2], stddev = 0.01)),
    'output' : tf.Variable(tf.random_normal([n_hidden2, n_output], stddev = 0.01))
}

biases = {
    'hidden1' : tf.Variable(tf.random_normal([n_hidden1], stddev = 0.01)),
    'hidden2' : tf.Variable(tf.random_normal([n_hidden2], stddev = 0.01)),
    'output' : tf.Variable(tf.random_normal([n_output], stddev = 0.01))
}

In [60]:
def Qnet(x, weights, biases):
    hidden1 = tf.add(tf.matmul(x, weights['hidden1']), biases['hidden1'])
    hidden1 = tf.nn.relu(hidden1)
    
    hidden2 = tf.add(tf.matmul(hidden1, weights['hidden2']), biases['hidden2'])
    hidden2 = tf.nn.relu(hidden2)
    
    output = tf.add(tf.matmul(hidden2, weights['output']), biases['output'])
    return output

In [61]:
Qpred = Qnet(x, weights, biases)
loss = tf.reduce_mean(tf.square(y - Qpred))

LR = 0.001
optm = tf.train.AdamOptimizer(LR).minimize(loss)

In [62]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [63]:
replay_buffer = []
n_buffer = 5000
n_batch = 10 #10개씩 샘플링하여 배치프로세스를 진행한다.

gamma = 0.9

for episode in range(2001):
    
    done = False
    state = env.reset()
    
    count = 0
    
    while not done:
        
        count += 1
        state = np.reshape(state, [1, 4])
        Q = sess.run(Qpred, feed_dict = {x:state})
        
        epsilon = 0.1
        if np.random.uniform() < epsilon:
            a = env.action_space.sample()
        else:
            a = np.argmax(Q)
            
        next_state, reward, done, _ = env.step(a)
        
        if done:
            reward = -200
#             Q[0][a] = reward
#             next_state = np.zeros(state.shape) # 다음 시뮬레이션에서 없음
        
        # 버퍼에 저장!
        replay_buffer.append([state, a, reward, next_state, done]) 
        
        if len(replay_buffer) > n_buffer:
            replay_buffer.pop(0)
        
        state = next_state
    
    if episode % n_batch == 1:
        for _ in range(50):
            # learning
            if len(replay_buffer) < n_batch:
                break

            minibatch = random.sample(replay_buffer, n_batch)

            x_stack = np.empty(0).reshape(0, n_input)
            y_stack = np.empty(0).reshape(0, n_output)

            for state, action, reward, next_state, done in minibatch:
                Q = sess.run(Qpred, feed_dict = {x: np.reshape(state, [1, n_input])})

                if done:
                    Q[0][action] = reward
                else:
                    next_Q = sess.run(Qpred, feed_dict = {x: np.reshape(next_state, [1, n_input])})
                    Q[0][action] = reward + gamma*np.max(next_Q)

                x_stack = np.vstack([x_stack, state])
                y_stack = np.vstack([y_stack, Q])

            sess.run(optm, feed_dict = {x: x_stack, y: y_stack})


    if episode % 200 == 1:
        print("Episode: {} steps: {}".format(episode, count))

env.close()

Episode: 1 steps: 10
Episode: 201 steps: 78
Episode: 401 steps: 121
Episode: 601 steps: 148
Episode: 801 steps: 56
Episode: 1001 steps: 500
Episode: 1201 steps: 201
Episode: 1401 steps: 500
Episode: 1601 steps: 500
Episode: 1801 steps: 500


In [76]:
state = env.reset()

done = False

while not done:
    env.render()
    
    state = np.reshape(state, [1, n_input])
    Q = sess.run(Qpred, feed_dict={x:state})
    action = np.argmax(Q)
    
    next_state, reward, done, _ = env.step(action)
    state = next_state
    
    
env.close()